## Conclusion:
- Pour le moement pas de raisons dans les données que l'on recupère qui influencerait un calcul de SoH plus bas
- Le SoH calculé est plus bas car l'energie gagnée par la batterie pendant la charge est inferieru a ce qu'elle eput recevoir pour une charge équivalente. 

In [ ]:
from transform.processed_tss.ProcessedTimeSeries import *
import pandas as pd
from core.pandas_utils import *
from transform.raw_results.config import *
import plotly.express as px
from transform.raw_results.tesla_results import get_results

In [ ]:
from transform.processed_results.main import get_processed_results

## Data Load

In [ ]:
df = get_processed_results('tesla')

In [ ]:
from core.sql_utils import *
engine = get_sqlalchemy_engine()
con = engine.connect()

with engine.connect() as connection:
    dbeaver_df = pd.read_sql(text("""SELECT * FROM vehicle_data vd
            join vehicle v
            on v.id = vd.vehicle_id
            join vehicle_model vm 
            on vm.id = v.vehicle_model_id
            join battery b
            on b.id = vm.battery_id
            WHERE vm.model_name like '%model%';"""), con)

dbeaver_df.head(3)

In [ ]:
df = df.merge(dbeaver_df.groupby('vin', as_index=False)["version"].first().rename(columns={'version':'tesla_code'}), on='vin', how='left')

In [ ]:
# on prend uniquement les MTY13 et les perits odomètres
df_mty13 = df[(df['tesla_code']=="MTY13") & (df['odometer'] < 45000)]

In [ ]:
processed_ts = TeslaProcessedTimeSeries("tesla", force_update=True)
processed_ts_mty13 = processed_ts[processed_ts['tesla_code']=='MTY13']
processed_ts_mty13_gb = processed_ts_mty13[processed_ts_mty13['trimmed_in_charge']==True].groupby(["vin", "trimmed_in_charge_idx"], observed=True, as_index=False).agg(
    soc_diff=pd.NamedAgg("soc", series_start_end_diff),
    energy_added_min=pd.NamedAgg("charge_energy_added", "min"),
    energy_added_end=pd.NamedAgg("charge_energy_added", "last"),
    capacity=pd.NamedAgg("capacity", 'last'),
    odometer=('odometer', 'last'),
    charging_power=pd.NamedAgg("charging_power", "median"),

) .eval("energy_added = energy_added_end - energy_added_min").eval("soh = energy_added / (soc_diff / 100.0 * capacity)")


## Constat

In [ ]:
px.scatter(df_mty13, y='soh', x='odometer', color='vin')

On constate un nuage de point en .94 

In [ ]:
px.scatter(processed_ts_mty13_gb[(processed_ts_mty13_gb["soh"] < 2) 
                                 & (processed_ts_mty13_gb["soh"] > .5) 
                                 & (processed_ts_mty13_gb["soc_diff"] > 40)], x='odometer', y='soh', color='vin')

On castate un nueage de points en .9 pour des vin qui ont moins de 50k km.

## Etude des vin

In [ ]:
from transform.raw_tss.tesla_raw_tss import get_raw_tss

In [ ]:
tss = get_raw_tss()
tss_mty13 = tss[tss['vin'].isin(df_mty13.vin.unique())]
tss_mty13_charging = tss_mty13[tss_mty13['charging_state']=='Charging'].copy()

### Vin avec des SoH hauts et bas

Liste des vin qui ont des valeurs hautes et basses avec mons de 50k km

In [ ]:
search = pd.DataFrame(processed_ts_mty13_gb[(processed_ts_mty13_gb["soc_diff"] > 40)].groupby('vin').agg(soh=('soh',list),
                                                                                                         odometer=('odometer', "max")))
search_filtered = search[search["soh"].apply(lambda x: len(x) >= 2)]

search_filtered['max'] = search_filtered["soh"].apply(lambda x: max(x))
search_filtered['min'] = search_filtered["soh"].apply(lambda x: min(x))
search_filtered[(search_filtered['max'] > .97) & (search_filtered['min'] < .91) & (search_filtered["odometer"] < 50000)]

On prend le vin LRWYGCFS0PC674888 qui a  des valeurs haute et basse pour le SoH

Graph 

In [ ]:
px.scatter(processed_ts_mty13_gb[(processed_ts_mty13_gb["soh"] < 2) 
                                 & (processed_ts_mty13_gb["soh"] > .5) 
                                 & (processed_ts_mty13_gb["soc_diff"] > 40)
                                 & (processed_ts_mty13_gb['vin']=="XP7YGCFS9RB393566")], x='odometer', y='soh', color='vin')

In [ ]:
(processed_ts_mty13_gb[(processed_ts_mty13_gb["soh"] < 2) 
                                 & (processed_ts_mty13_gb["soh"] > .5) 
                                 & (processed_ts_mty13_gb["soc_diff"] > 40)
                                 & (processed_ts_mty13_gb['vin']=="XP7YGCFS9RB393566")])

In [ ]:
processed_ts_mty13[(processed_ts_mty13['trimmed_in_charge']==True) & 
                   (processed_ts_mty13['vin']=="XP7YGCFS9RB393566") & 
                   (processed_ts_mty13['trimmed_in_charge_idx'].isin([71]))].describe()[[
                       'soc','battery_range', 'charge_current_request', 'charge_current_request_max','charge_energy_added', 
                       'charge_limit_soc', 'charge_limit_soc_min','charge_miles_added_ideal', 'charge_miles_added_rated', 
                       'charging_rate','charger_actual_current', 'charger_pilot_current', 'charging_power','charger_voltage', 
                       'est_battery_range', 'odometer', 'inside_temp','outside_temp', 'time_diff', 'capacity']]

In [ ]:
processed_ts_mty13[(processed_ts_mty13['trimmed_in_charge']==True) & 
                   (processed_ts_mty13['vin']=="XP7YGCFS9RB393566") & (processed_ts_mty13['trimmed_in_charge_idx'].isin([203,]))].describe()[['soc',
       'battery_range', 'charge_current_request', 'charge_current_request_max',
        'charge_energy_added', 'charge_limit_soc', 'charge_limit_soc_min',
       'charge_miles_added_ideal', 'charge_miles_added_rated', 'charging_rate',
       'charger_actual_current', 'charger_pilot_current', 'charging_power',
       'charger_voltage', 'est_battery_range', 'odometer', 'inside_temp',
       'outside_temp', 'time_diff', 'capacity']]

Pas de différence entre les deux charges sur les paramètres ext accessibles, pourtant un SOH bien différent. 

### Vin avec un SoH uniquement bas

Si on prend un vin qui a uniquemnt des valeurs de SoH basses

vin avec uniquement des valeurs basses par exemple  "XP7YGCES0SB552093", 'XP7YGCES0SB554152', 'XP7YGCES6SB537145', 'XP7YGCES5RB500470'

In [ ]:
(processed_ts_mty13_gb[(processed_ts_mty13_gb["soh"] < 2) 
                                 & (processed_ts_mty13_gb["soh"] > .5) 
                                 & (processed_ts_mty13_gb["soc_diff"] > 40)
                                 & (processed_ts_mty13_gb['vin']=="XP7YGCES0SB552093")])

In [ ]:
processed_ts_mty13[(processed_ts_mty13['trimmed_in_charge']==True) & 
                   (processed_ts_mty13['vin']=="XP7YGCES0SB552093") & (processed_ts_mty13['trimmed_in_charge_idx'].isin([137,]))].describe()[['soc',
       'battery_range', 'charge_current_request', 'charge_current_request_max',
        'charge_energy_added', 'charge_limit_soc', 'charge_limit_soc_min',
       'charge_miles_added_ideal', 'charge_miles_added_rated', 'charging_rate',
       'charger_actual_current', 'charger_pilot_current', 'charging_power',
       'charger_voltage', 'est_battery_range', 'odometer', 'inside_temp',
       'outside_temp', 'time_diff', 'capacity']]

In [ ]:
processed_ts_mty13[(processed_ts_mty13['trimmed_in_charge']==True) & 
                   (processed_ts_mty13['vin']=="XP7YGCES0SB552093") & (processed_ts_mty13['trimmed_in_charge_idx'].isin([101,]))].describe()[['soc',
       'battery_range', 'charge_current_request', 'charge_current_request_max',
        'charge_energy_added', 'charge_limit_soc', 'charge_limit_soc_min',
       'charge_miles_added_ideal', 'charge_miles_added_rated', 'charging_rate',
       'charger_actual_current', 'charger_pilot_current', 'charging_power',
       'charger_voltage', 'est_battery_range', 'odometer', 'inside_temp',
       'outside_temp', 'time_diff', 'capacity']]

Pas de différence sur les paramètre ext avec une charge qui donne un SoH plus élevé.